You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [8]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [9]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [11]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [12]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [13]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [15]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of artificial intelligence and its impact on various industries
- Mention of the rapid advancements and innovations in AI technology

Key Points:
1. Latest Trends in Artificial Intelligence:
- Deep learning and neural networks
- Natural language processing and sentiment analysis
- Autonomous vehicles and robotics
- Healthcare applications such as medical imaging and predictive analytics
- AI et

- Display the results of your execution as markdown in the notebook.

In [18]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial intelligence (AI) has become a transformative force across various industries, revolutionizing the way we live and work. The rapid advancements in AI technology have led to exciting developments that are reshaping the future of society. From deep learning and neural networks to natural language processing and autonomous vehicles, AI is at the forefront of innovation. In this blog post, we will explore the latest trends and key players in the AI industry, shedding light on the impact of AI on society and the ethical considerations surrounding this technology.

## Latest Trends in Artificial Intelligence

### Deep Learning and Neural Networks
Deep learning has emerged as a powerful subset of machine learning, enabling computers to learn from data and make decisions without human intervention. Neural networks, inspired by the human brain, are at the core of deep learning algorithms, driving advancements in image and speech recognition, natural language processing, and more.

### Natural Language Processing and Sentiment Analysis
Natural language processing (NLP) allows computers to understand and generate human language, leading to applications like chatbots, language translation, and sentiment analysis. Sentiment analysis, in particular, helps businesses gauge public opinion and customer feedback, making it a valuable tool for market research.

### Autonomous Vehicles and Robotics
AI is revolutionizing the transportation industry with the development of autonomous vehicles and drones. Self-driving cars equipped with AI technology are poised to enhance road safety, reduce traffic congestion, and transform the way we commute. Robotics, powered by AI, are also making significant strides in manufacturing, healthcare, and other sectors.

### Healthcare Applications
In the healthcare sector, AI is being leveraged for medical imaging, predictive analytics, and personalized medicine. AI-powered tools can analyze medical images with precision, predict patient outcomes, and recommend personalized treatment plans, improving healthcare delivery and patient outcomes.

### AI Ethics and Bias Mitigation
As AI technology becomes more pervasive, concerns about ethics and bias have come to the forefront. Ensuring that AI systems are fair, transparent, and accountable is crucial to building trust and mitigating potential harm. Ethical considerations, such as data privacy, algorithmic bias, and the impact of AI on job displacement, are key areas of focus for policymakers and industry stakeholders.

## Key Players in the AI Industry

The AI industry is populated by key players who are driving innovation and shaping the future of AI technology. Companies like Google, IBM, Microsoft, Amazon, Tesla, Facebook, OpenAI, and NVIDIA are at the forefront of AI research and development. Google's research division, Google Brain, and its subsidiary DeepMind are leading the way in AI breakthroughs. IBM's Watson platform is revolutionizing industries with its cognitive computing capabilities. Microsoft's Azure AI services are empowering businesses with AI solutions. Amazon's Alexa has become a household name for voice-activated AI assistants. Tesla's Autopilot feature is paving the way for autonomous driving. Facebook's AI Research (FAIR) lab is pushing the boundaries of AI research. OpenAI is dedicated to ensuring that AI benefits all of humanity. NVIDIA's GPUs are powering AI applications across industries, from gaming to healthcare.

## Noteworthy News in Artificial Intelligence

### Recent Breakthroughs in AI Research
AI research has seen significant breakthroughs in recent years, from AlphaGo's victory over human Go champions to GPT-3's language generation capabilities. These advancements are pushing the boundaries of AI capabilities and opening up new possibilities for AI applications.

### Acquisitions and Partnerships in the AI Industry
Mergers and acquisitions are reshaping the AI landscape, as companies seek to bolster their AI capabilities through strategic partnerships. From tech giants acquiring AI startups to industry collaborations, the AI industry is witnessing a wave of consolidation and collaboration.

### Impact of AI on Job Market and Workforce
The rise of AI technology has sparked debates about its impact on the job market and the future of work. While AI has the potential to automate repetitive tasks and enhance productivity, there are concerns about job displacement and the need for upskilling and reskilling to adapt to a changing workforce landscape.

### Ethical Concerns and Regulations Surrounding AI Technology
Ethical considerations are paramount in the development and deployment of AI technology. Issues like data privacy, algorithmic bias, transparency, and accountability are central to ensuring that AI systems are fair and beneficial for society. Policymakers and industry leaders are grappling with the need for regulations to govern the ethical use of AI.

## Conclusion

In conclusion, artificial intelligence is a dynamic field that is reshaping industries and societies worldwide. The latest trends in AI, from deep learning to autonomous vehicles, are driving innovation and transforming the way we live and work. Key players in the AI industry, such as Google, IBM, Microsoft, and Amazon, are at the forefront of AI research and development. Noteworthy news in AI, including breakthroughs in research, acquisitions, and ethical concerns, are shaping the future of AI technology. As we navigate the complexities of AI ethics and regulations, it is essential to stay informed and engaged in discussions about the impact of AI on society. By exploring further resources and articles on AI trends and sharing our thoughts and experiences with AI technology, we can shape a future where AI benefits all of humanity.